# Particle Swarm Optimisation for Knights Covering Problem

In [2]:
#necessary python library imports
import numpy as np
from collections.abc import Callable

## Algorithm Implementation

### Particle Swarm Optimisation class

Fitness function is an error function, so we want to minimise it

In [2]:
class PSO:
    def __init__(self, board_size: int, fitness_function: Callable[[np.ndarray], np.ndarray], 
                 penalty_function: Callable[[np.ndarray, int], np.ndarray] = (lambda _: 0), num_particles: int = 100, c1: float = 1.0, 
                 c2: float = 1.0, max_velocity: float = 4.0, min_velocity: float = -4.0, inertia: float = 1.0, rng_seed: int = None) -> None:
        
        #check for sensible hyperparameter selection
        assert min_velocity < max_velocity, "Max velocity has to be greater than the min velocity." 
        assert num_particles > 1, "There has to be at least one particle"
        assert c1 >= 0, "c1 has to be greater than 0"
        assert c2 >= 0, "c2 has to be greater than 0"
        assert board_size > 3, "board size has to be at least 3"
        #condition on sensible inertia values as proposed in: "F. van den Bergh. An Analysis of Particle Swarm Optimizers (2002)"
        assert inertia > c1+c2 / 2 - 1 and inertia >= 0, "Inertia must be positive and greater than (c1 + c2) / 2 -1"

        #assign hyperparameters to object variables
        self.board_size = board_size
        self.fitness_function = fitness_function
        self.penalty_function = penalty_function
        self.num_particles = num_particles
        self.c1 = c1
        self.c2 = c2
        self.max_velocity = max_velocity
        self.min_velocity = min_velocity
        self.inertia = inertia
        self.rng = np.random.default_rng(rng_seed)

        #initialise particles and velocities
        self.particle_positions = self.rng.integers(low=0, high=1, endpoint=True, size=(self.num_particles, self.board_size**2))
        self.particle_best_pos = self.particle_positions.copy()
        self.particle_velocities = self.rng.random(size=(self.num_particles, self.board_size**2)) * self.max_velocity
        
        #initialise global best
        self.particle_fitness = self.compute_penalty_fitness(self.particle_positions, 0)
        self.particle_best_fitness = self.particle_fitness.copy()

        best_index = np.argmin(self.particle_fitness)
        self.global_best_location = self.particle_positions[best_index].copy()
        self.global_best_fitness = self.particle_fitness[best_index]
    
    def compute_penalty_fitness(self, particle_positions: np.ndarray, k: int):
        ''' Computes the fitness subtracted by the penalty for invalid solutions (i.e. not all squares attacked or occupied).
            The penalty function receives a parameter 'k', which indicates the current iteration number.
        '''
        return self.fitness_function(particle_positions) + self.penalty_function(particle_positions, k)

    def optimise(self, num_iterations = 10_000):
        ''' Starts the optimisation loop. The num_iterations parameter specifies the terimination criterion as 
            the number of iterations that will be done.
        '''

        # initialise random numbers used in velocity computation
        r1 = np.empty(shape=self.num_particles)
        r2 = np.empty(shape=self.num_particles)
        r_sigmoid = np.empty(shape=self.num_particles)

        for i in range(num_iterations):
            #update random numbers
            self.rng.random(size=self.num_particles, out=r1)
            self.rng.random(size=self.num_particles, out=r2)
            self.rng.random(size=self.num_particles, out=r_sigmoid)

            # update velocities
            reduced_current_velocity = self.inertia * self.particle_velocities
            local_velocity_component = self.c1 * r1 * (self.particle_best_pos - self.particle_positions)
            global_velocity_component = self.c2 * r2 * (self.global_best_location - self.particle_positions)

            new_velocity = reduced_current_velocity + local_velocity_component + global_velocity_component
            self.particle_velocities = np.clip(new_velocity, a_max= self.max_velocity, a_min= self.min_velocity)

            # update positions
            #the original algorithm was flawed in regards to the position update
            #see https://www.researchgate.net/publication/224302958_A_novel_binary_particle_swarm_optimization
            sigmoid = 1/(1 + np.exp(-self.particle_velocities))
            change_indices = r_sigmoid < sigmoid

            #dimensions in which the normalised velocity value is greater than random value change their location value from 0 to 1 or vice versa
            self.particle_positions[change_indices] == (self.particle_positions[change_indices] + 1) % 2

            # calculate fitness
            self.particle_fitness = self.compute_penalty_fitness(self.particle_positions, i)

            # update best positions
            # local best positions
            new_best_local_fitness_indices = self.particle_fitness > self.particle_best_fitness
            self.particle_best_fitness[new_best_local_fitness_indices] = self.particle_fitness[new_best_local_fitness_indices]
            self.particle_best_pos[new_best_local_fitness_indices] = self.particle_positions[new_best_local_fitness_indices]

            # global best positions
            new_best_global_fitness_index = np.argmin(self.particle_fitness)
            new_best_global_fitness = self.particle_fitness[new_best_global_fitness_index]

            #update best solution 
            if new_best_global_fitness < self.global_best_fitness:
                self.global_best_fitness = new_best_global_fitness
                self.global_best_location = self.particle_positions[new_best_global_fitness].copy()
            


### Fitness and penalty functions

#### Utility functions

In [61]:
def compute_covered_squares(particle_position: np.ndarray) -> np.ndarray:
    ''' 
    Computes all covered squares given the position of a single particle. 
    Covered squares are defined as positions on the chessboard that are either occupied or attacked by a knight.
    '''
    board_size = int(np.sqrt(particle_position.shape[0]))
    knight_position_indices = np.argwhere(particle_position.reshape((board_size, board_size)))

    covered_positions = []
    relative_jump_positions = [
        #(x_change, y_change)
        ( 0, 0),#current position
        (-1,-2),#left top
        (+1,-2),#left bottom
        (-1,+2),#right top
        (+1,+2),#right bottom
        (-2,-1),#top left
        (-2,+1),#top right
        (+2,-1),#bottom left
        (+2,+1),#bottom right
    ]

    for relative_jump_position in relative_jump_positions:
        #calculate positions that the knights can jump to
        positions = knight_position_indices.copy()
        positions += relative_jump_position   

        #calculated positions are only valid if they are within the bounds of the chessboard     
        valid_indices = np.all((positions >= 0) & (positions < board_size), axis=1)

        #append valid positions to the list of attacked positions
        covered_positions.extend(positions[valid_indices])
    
    covered_positions = np.array(covered_positions)
    unique_positions = np.unique(covered_positions, axis=0)

    return unique_positions

def compute_num_of_covered_squares(particle_position: np.ndarray) -> int:
    return compute_covered_squares(particle_position).shape[0]


a = np.array([0,1,1,0,0, 
              0,0,0,0,0, 
              0,0,1,0,1, 
              0,1,0,0,0,
              1,0,0,0,0,])
covered_squares = compute_covered_squares(a)
print(covered_squares)

#### Fitness function implementations

In [62]:
def fitness_binary_space_pso_paper(particle_positions: np.ndarray) -> np.ndarray:
    '''From paper Investigating binary PSO parameter influence on the knights cover problem by N. Franken and A.P. Engelbrecht (2005)'''
    num_covered_squares = np.apply_along_axis(compute_num_of_covered_squares, axis=1, arr = particle_positions)
    num_knights = np.sum(particle_positions, axis=1)
    total_num_squares = particle_positions.shape[1]
    num_empty_squares = total_num_squares - num_covered_squares

    fitness = num_empty_squares + num_knights + total_num_squares/num_covered_squares
    return fitness

def fitness_integer_space_pso_paper(particle_positions: np.ndarray) -> np.ndarray:
    pass

def fitness_binary_space_simple(particle_positions: np.ndarray) -> np.ndarray:
    pass

def fitness_integer_space_simple(particle_positions: np.ndarray) -> np.ndarray:
    pass

#### Penalty function implementations

In [3]:
#computes penalty for conventional search space depending on the ratio of empty squares
def penalty_binary_space(particle_positions: np.ndarray, iteration_number: int):
    ratio_of_empty_squares = (particle_positions.shape[1] - compute_num_of_covered_squares(particle_positions)) / particle_positions.shape[1]

    penalties = iteration_number * ratio_of_empty_squares #use ratio so that the penalty is independent of the board size
    return penalties

#computes penalty for integer search space depending on ratio of empty squares and whether the knights are positioned outside the board
def penalty_integer_space(particle_positions:np.ndarray, iteration_number: int, board_size):
    ratio_of_empty_squares = (board_size**2 - compute_num_of_covered_squares(particle_positions)) / board_size**2
    last_knight_location = np.sum(particle_positions, axis=1)
    distance_from_last_square = last_knight_location - board_size**2

    penalties = iteration_number * (ratio_of_empty_squares + np.maximum(0, distance_from_last_square))
    return penalties

### Random Search Algorithm class

In [ ]:
class RandomSearch:
    def __init__(self, board_size: int, fitness_function: Callable[[np.ndarray], np.ndarray], 
                 penalty_function: Callable[[np.ndarray, int], np.ndarray] = (lambda _: 0), rng_seed: int = None):
        
        self.board_size = board_size
        self.fitness_function = fitness_function
        self.penalty_function = penalty_function
        self.rng = np.random.default_rng(rng_seed)

        self.current_solution = self.rng.integers(low=0, high=1, endpoint=True, size=board_size**2)
        self.best_solution = self.current_solution.copy()
        self.best_fitness = self.compute_penalty_fitness(self.best_solution, 0)
        
    def compute_penalty_fitness(self, particle_positions: np.ndarray, k: int):
        ''' Computes the fitness subtracted by the penalty for invalid solutions (i.e. not all squares attacked or occupied).
            The penalty function receives a parameter 'k', which indicates the current iteration number.
        '''
        return self.fitness_function(particle_positions) + self.penalty_function(particle_positions, k)
        
    def optimise(self, num_iterations= 10_000):
        for i in range(num_iterations):
            #select random position to be altered
            mutation_index = self.rng.integers(low=0, high=self.current_solution.shape[0])
            #set element at index position to it's binary complement
            self.current_solution[mutation_index] = (self.current_solution[mutation_index]+1) % 2 
            #compute fitness of altered solution
            fitness = self.compute_penalty_fitness(self.current_solution, i)
            
            #update best solution
            if(fitness < self.best_fitness):
                self.best_solution = self.current_solution.copy()
                self.best_fitness = fitness

### Stochastic Hillclimb Algorithm class

In [4]:
#implements First choice stochastic hill climb as per Artificial Intelligence: A Modern Approach, Global Edition, Fourth Edition
class StochasticHillclimb:
    def __init__(self, board_size: int, fitness_function: Callable[[np.ndarray], np.ndarray], 
                 penalty_function: Callable[[np.ndarray, int], np.ndarray] = (lambda _: 0), allowed_plateau_steps: int = 100, rng_seed: int = None):
        
        self.board_size = board_size
        self.fitness_function = fitness_function
        self.penalty_function = penalty_function
        self.rng = np.random.default_rng(rng_seed)
        self.allowed_plateau_steps = allowed_plateau_steps

        self.current_solution = self.rng.integers(low=0, high=1, endpoint=True, size=board_size**2)
        self.current_fitness = self.compute_penalty_fitness(self.current_solution, 0)

    def compute_penalty_fitness(self, particle_positions: np.ndarray, k: int):
        ''' Computes the fitness subtracted by the penalty for invalid solutions (i.e. not all squares attacked or occupied).
            The penalty function receives a parameter 'k', which indicates the current iteration number.
        '''
        return self.fitness_function(particle_positions) + self.penalty_function(particle_positions, k)

    def optimise(self, num_iterations= 10_000):
        num_plateau_steps = 0

        for i in range(num_iterations):
            neighbour_permutation = self.rng.permutation(self.current_solution.shape[0])

            for neighbour_index in neighbour_permutation:
                neighbour_solution = self.current_solution.copy()
                neighbour_solution[neighbour_index] = (neighbour_solution[neighbour_index]+1) % 2 
                neighbour_fitness = self.compute_penalty_fitness(neighbour_fitness, i)

                #choose first neighbour with more optimal fitness (in this case, lower fitness value as we want to minimise the function)
                if neighbour_fitness <= self.current_fitness:
                    #check if fintess is on a plateau
                    if neighbour_fitness == self.current_fitness:
                        num_plateau_steps += 1 #increment plateau step counter

                        #cancel optimisation if maximum number of plateau steps is reached
                        if num_plateau_steps > self.allowed_plateau_steps: return
                        
                    else: #reset plateau step counter if fitness is reduced
                        num_plateau_steps = 0

                    self.current_solution = neighbour_solution
                    self.current_fitness = neighbour_fitness
                    break #dont consider the remaining neighbours
            
                return # no improved solution was found among the neighbours -> peak has been reached

## Experimentation

### Utility functions

### Experimentation config

### Run experiments

## Results

### Load results file

### Visualisation